# 영화 리뷰 데이터 웹 크롤링


### (Web Crawling for Movie Reviews) 

1. 페이지의 리뷰 모두 저장하기
2. 여러 페이지의 리뷰 저장하기
3. 영화의 전체 리뷰 모두 저장하기

In [1]:
# 웹 크롤링에 필요한 모듈 불러오기

from urllib.request import urlopen
from bs4 import BeautifulSoup

## 1. 페이지의 리뷰  모두 저장하기

In [2]:
# urlopen()을 활용해 웹 상의 데이터 받아오기

url = 'http://movie.daum.net/moviedb/grade?movieId=109512&type=netizen&page=1'
webpage = urlopen(url)

In [3]:
source = BeautifulSoup(webpage, 'html5lib')
reviews = source.find_all('p', {'class':'desc_review'})
for review in reviews:
    print(review.get_text().strip())

조선족범죄자들 일망타진실화 살벌.잔인하고 유쾌하지않지만 통쾌했다
마블리는 38사기동대와 비슷한 연기와 가오란 가오는 엄청 잡는 형사.
윤계상은 착하게 생겨서 악역이 어울리지 않는 느낌..
웃음포인트가 뻔한 그냥 조폭영화고 좀 잔인한 장면은 눈 감고 봄
하얼빈 삼총사에게 박수를!!
하도 알바들 설쳐대서 안볼까하다가 오늘 봤다. 조연들이 살린 영화. 윤계상은 악역 안어울려. 마동석은 덩치값만 하네요.
연기가 조금 부자연스런점 빼고는 나름 괜찮은영화
윤계상.. 아무리봐도 " 나 연기 하고 있어~~ " 하던데.. 
나는 아무리 잘 줘도 7점이네.
마동석ㅎㅎ 윤계상삼총사무서웠어요
잔인하기도했지만재밌게봤어요

청년경찰만큼재밌던영화다
좀잔인해도가려서보면된다
마동석 윤계상흥해라~♡
흔한조폭영화


## 2. 여러 페이지의 리뷰 저장하기

In [4]:
#리뷰 내용을 담기 위한 리스트 생성
review_list = []

for n in range(5):
    url = 'http://movie.daum.net/moviedb/grade?movieId=109512&type=netizen&page={}'.format(n+1)
    webpage = urlopen(url) 
    source = BeautifulSoup(webpage, 'html5lib')
    reviews = source.find_all('p', {'class':'desc_review'})
    for review in reviews:
        review_list.append(review.get_text().strip())

In [5]:
# data.txt 파일로 리뷰 내용 출력하기

file = open('data.txt','w', encoding = 'utf-8')

for review in review_list:
    file.write(review + '\n')

file.close()

## 3. 영화의 전체 리뷰 모두 저장하기

In [6]:
# 페이지 개수 계산하기
url = 'http://movie.daum.net/moviedb/grade?movieId=109512&type=netizen&page=1'
webpage = urlopen(url)
source = BeautifulSoup(webpage, 'html5lib')
review = source.find('span', {'class':'num_review'})

In [7]:
t = review.get_text().strip()
t = t[:-1].replace(',','')
page_no = int(int(t[10:])/10)

# 위의 공식 상세 풀이.
#t = review.get_text().strip()  # 공백을 지움.
#t1 = t[:-1]                    # 맨 뒤 )를 지움
#t2 = t1.replace(',','')        # 숫자에 있는 천 단위 , 를 지움
#t3 = t2[10:]                   # 앞에를 다 지워서 숫자만 남김 > 문자형(str) 상태
#t4 = int(t3)/10                # 한 페이지 당 리뷰가 10개씩 기록 되므로 10개를 나눔 
#page_no = int(t4)              # 정수로 변환해 총 페이지 개수를 계산

In [8]:
review_list = []
for n in range(page_no):
    url = 'http://movie.daum.net/moviedb/grade?movieId=109512&type=netizen&page={}'.format(n+1)
    webpage = urlopen(url)
    source = BeautifulSoup(webpage, 'html5lib')
    reviews = source.find_all('p', {'class':'desc_review'})
    for review in reviews:
        review_list.append(review.get_text().strip())

In [9]:
file = open('theoutlaws.txt', 'w', encoding='utf-8')
for review in review_list:
    file.write(review + '\n')
file.close()

#  데이터 전처리 & 데이터 탐색

-KoNLPy 이용하여 리뷰 데이터 분석하기, 자주 사용되는 단어 추출하기

In [10]:
# 크롤링한 영화 리뷰 데이터 불러오기
f = open('theoutlaws.txt', encoding='utf-8')
lines = f.readlines()
f.close()

In [11]:
# 한국어 텍스트 분석에 필요한 트위터 형태소 분석기 불러오기(konlpy)
from konlpy.tag import Twitter
tw = Twitter()

In [12]:
# 전체 텍스트의 형태소를 분석
sentences_tag = []
for x in lines: 
    sentences_tag.append(tw.pos(x))

In [21]:
# sentences_tag의 내용물을 출력
# print(sentences_tag)
print(len(sentences_tag))

2652


In [14]:
# 리뷰를 분석하는데 있어 중요한 품사인 명사와 형용사만 추출
        
keywords = []                                 # 명사, 형용사만 담기 위한 리스트 생성
for x in sentences_tag:                       # sentences_tag의 list에서 for문으로 하나씩 꺼냄
    for word, tag in x:                       # 각 tuple의 (word, tag)를 for문으로 각각 꺼냄
        if tag in ['Noun', 'Adjective']:      # tag가 Noun(명사) or Adjective(형용사)이면 keywords의 list에 append 함.
            keywords.append(word)

In [15]:
# 가장 많이 사용된 명사와 형용사 출력하기

from collections import Counter            # collection 패키지에서 Counter(등장 횟수를 세주는 역할)를 불러옴
counts = Counter(keywords)                 # keywords에서 서로 다른 명사와 형용사가 얼마나 많이 쓰였는지를 셈
print(counts.most_common(20))              # 20개의 가장 많이 사용된 단어를 추출

[('영화', 633), ('마동석', 407), ('윤계상', 388), ('연기', 368), ('잔인', 187), ('좋', 182), ('너무', 160), ('배우', 159), ('최고', 132), ('시간', 122), ('액션', 116), ('진짜', 107), ('정말', 105), ('조연', 85), ('보고', 85), ('재미', 85), ('범죄', 75), ('통쾌', 68), ('장면', 67), ('그냥', 64)]


# 데이터 시각화(워드클라우드)

In [16]:
import pytagcloud, random, webbrowser       # pytagcloud 모듈 사용
from konlpy.tag import Twitter              # 트위트 형태소 분석기 사용
from collections import Counter             # collections 패키지에서 counter 불러옴.

In [17]:
# 워드클라우드 생성에 필요한 함수 정의

def get_tags(text, ntags = 100, multiplier = 1):
    t = Twitter()
    nouns = []
    for s in text:
        for noun in t.nouns(s):
            if noun in ['영화']:
                pass
            else:
                nouns.append(noun)
    count = Counter(nouns)
    return [{'color':color(), 'tag':n, 'size': c * multiplier} \
            for n, c in count.most_common(ntags)]

In [18]:
def draw_cloud(tags, filename, fontname = 'Noto Sans CJK', size = (1500, 800)):
    pytagcloud.create_tag_image(tags, filename, fontname = fontname, size = size) 
    webbrowser.open(filename)

In [19]:
# 단어 구분을 위한 색 추출
r = lambda: random.randint(0,255)   # 람다식을 통해 0부터 254 사이의 임의의 정수 추출
color = lambda: (r(), r(), r())     # 람다식 3개를 합쳐 , RGB로 색을 정의

In [20]:
# 데이터를 불러와 워드클라우드 그리기
f = open('theoutlaws.txt', encoding='utf-8')
lines = f.readlines()
f.close()

tags = get_tags(lines)
draw_cloud(tags, 'theoutlaws-wordcloud.png')